In [1]:
from DICOMLib import DICOMUtils
import JupyterNotebooksLib as slicernb
import slicer
import vtk
import sys
import numpy as np

In [2]:
dicomData = '/home/sliceruser/work/Documents/dicoms/native'
nodes = []

with DICOMUtils.TemporaryDICOMDatabase() as db:
    DICOMUtils.importDicom(dicomData, db)
    patientUIDs = db.patients()
    for patient in patientUIDs:
        nodes.extend(DICOMUtils.loadPatientByUID(patient))
volumeNode = getNode(nodes[0])
tumorNode = slicer.util.loadVolume('resources/Segmentation.nrrd')

Switching to temporary DICOM database: /tmp/Slicer-/20220327_122538_TempDICOMDatabase
Loading with imageIOName: GDCM
Window/level found in DICOM tags (center=40.0, width=400.0) has been applied to volume 3: Body 0.5 Venous-Phase CE


In [3]:
tumorVoxels = slicer.util.arrayFromVolume(tumorNode)

In [4]:
voxels = slicer.util.arrayFromVolume(volumeNode)

In [5]:
boneVoxels = (((voxels >= 300) & (voxels <= 1900)) * 1)

In [6]:
sphere = vtk.vtkSphereSource()
sphere.SetCenter(0,0,-800)
sphere.SetRadius(10)
model = slicer.modules.models.logic().AddModel(sphere.GetOutputPort())

In [7]:
volRenLogic = slicer.modules.volumerendering.logic()
displayNode = volRenLogic.CreateDefaultVolumeRenderingNodes(tumorNode)
volRenLogic.SetDefaultRenderingMethod("VTKGPURayCast")
displayNode.SetVisibility(True)
# displayNode.GetVolumePropertyNode().Copy(volRenLogic.GetPresetByName('MR-Default'))


In [8]:
model.GetDisplayNode().SetOpacity(0.4)
model.GetDisplayNode().SetColor(1,0,1)

In [9]:
slicernb.AppWindow.setWindowSize(scale=1)

In [10]:
app = slicernb.AppWindow()
app

In [11]:
cm = vtk.vtkCenterOfMass()
p = np.where(tumorVoxels != 0)
points = vtk.vtkPoints()
for i in range(len(p[0])):
    points.InsertNextPoint(p[0][i], p[1][i], p[2][i])


polyData = vtk.vtkPolyData()
polyData.SetPoints(points)
cm.SetInputData(polyData)
cm.SetUseScalarsAsWeights(False)
cm.Update()
centerOfTumor = list(cm.GetCenter())
ijk_ras = vtk.vtkMatrix4x4()
tumorNode.GetIJKToRASMatrix(ijk_ras)
Ras_center = ijk_ras.MultiplyFloatPoint([*centerOfTumor, 1])
sphere.SetCenter(Ras_center[:-1])